# Validación de la estimación de recursos

In [1]:
import modulo_conn_sql as mcq
import numpy as np
import pandas as pd 
import datetime 
import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter
import seaborn as sns
import sqlalchemy as sa
import urllib

def conectarSQL():
    conn = mcq.ConexionSQL()
    cursor = conn.getCursor()
    return cursor

#Query BD SQL-Server Cemex
def querySQL(query, parametros):
    #Conectar con base sql y ejecutar consulta
    cursor = conectarSQL()
    try:
        cursor.execute(query, parametros)
        #obtener nombre de columnas
        names = [ x[0] for x in cursor.description]
        
        #Reunir todos los resultado en rows
        rows = cursor.fetchall()
        resultadoSQL = []
            
        #Hacer un array con los resultados
        while rows:
            resultadoSQL.append(rows)
            if cursor.nextset():
                rows = cursor.fetchall()
            else:
                rows = None
                
        #Redimensionar el array para que quede en dos dimensiones
        resultadoSQL = np.array(resultadoSQL)
        resultadoSQL = np.reshape(resultadoSQL, (resultadoSQL.shape[1], resultadoSQL.shape[2]) )
    finally:
            if cursor is not None:
                cursor.close()
    return pd.DataFrame(resultadoSQL, columns = names)

In [105]:
#Variables 
pais='Colombia'
iteracion = "1"
#rangos de fecha
inicio_historia= datetime.date(2022,2,1)
#inicio_historia = fin_historia - datetime.timedelta(30 * 2) - MonthBegin(1)
#fin_historia = datetime.date.today()
fin_historia = datetime.date(2022,2,28)


version_forecast = 'PRECIERRE_FEB_2022'

lista_materiales = ['ADITIVO', 'ARENA', 'Agua', 'CEMENTO', 'CENIZA', 'GRAVA']

In [106]:
#Dataset de los consumos
df = querySQL( "SELECT * FROM AT51_Z1045_CONSU_TICKET2 WHERE FechaInicio between ? and ?" , (inicio_historia.strftime("%Y-%m-%d"), fin_historia.strftime("%Y-%m-%d")) )
df['year_month'] = df.FechaInicio.dt.to_period('M')
df['TipoMaterial'] = df['TipoMaterial'].str.strip()
df['TextoBreveMaterial'] = df['TextoBreveMaterial'].str.strip()
df['CantidadReal'] = df['CantidadReal'].astype(float)

#Dataset de los despachos
df_serv = querySQL( "{CALL SCAC_AP10_dataset_servicios_rango (?,?,?)}" , (pais, inicio_historia.strftime("%Y-%m-%d"), fin_historia.strftime("%Y-%m-%d")) )
df_serv = df_serv.fillna(value=np.nan)
df_serv['year_month'] = df_serv.FechaEntrega.dt.to_period('M')

In [107]:
#Dataset desagregacion materias primas
desagregacion = querySQL( "SELECT * FROM SCAC_AT40_desagregacion_materiales WHERE FechaEntrega BETWEEN ? AND ? AND Version = ? and Iteraciones = ?" , (inicio_historia.strftime("%Y-%m-%d"), fin_historia.strftime("%Y-%m-%d"), version_forecast, iteracion ) )


In [108]:
# eliminacion de duplicados

entregas = df.groupby(['Entrega', 'TipoMaterial', 'UnidadMedida', 'Material', 'TextoBreveMaterial'])['CantidadReal'].mean().reset_index()

#Recategorizacion de materiales
entregas['TipoMaterial2'] = np.select(
    [
        (entregas['TextoBreveMaterial'].str.contains('CEMENTO,BLANCO')) | (entregas['TextoBreveMaterial'].str.contains('CEM ART')),
        (entregas['TextoBreveMaterial'].str.contains('CENIZA')),
        (entregas['TextoBreveMaterial'].str.contains('GRAVA')),
        (entregas['TextoBreveMaterial'].str.contains('ARENA')),
        (entregas['TipoMaterial'].str.contains('ADI')),
        (entregas['TipoMaterial'].str.contains('ADC')),
        (entregas['TipoMaterial'].str.contains('FIB')),
        (entregas['TipoMaterial'].str.contains('AGU'))
    ],
    [
        
        'CEMENTO',
        'CENIZA',
        'GRAVA',
        'ARENA',
        'ADITIVO',
        'OTROS',
        'FIBRA',
        'Agua'
    ], default = 'OTROS'#entregas['TextoBreveMaterial']
)

#pivoteo de materiales para que cada uno quede en una columna
df2 = pd.pivot_table(
    entregas,
    index = ['Entrega'],
    columns =['TipoMaterial2'],
    values = ['CantidadReal'],
    aggfunc = np.mean
).reset_index().set_axis(['Entrega', 'real_ADITIVO', 'real_ARENA', 'real_Agua', 'real_CEMENTO', 'real_CENIZA', 'real_FIBRA', 'real_GRAVA', 'real_OTROS'], axis=1, inplace=False)

In [96]:
#Ultima paso para eliminar duplicados
dfmateriales = pd.merge(df_serv[['FechaEntrega', 'Entrega', 'Planta']], df2, on=['Entrega'] )

#se agrupan los consumos por dia y planta
real = dfmateriales.groupby(['FechaEntrega','Planta'])['real_ADITIVO', 'real_ARENA', 'real_Agua', 'real_CEMENTO', 'real_CENIZA', 'real_FIBRA', 'real_GRAVA', 'real_OTROS'].sum().reset_index()

#se une forast y real
validacion = pd.merge(real, desagregacion, left_on=['FechaEntrega', 'Planta'], right_on=['FechaEntrega', 'PlantaUnica'])

#calculo de errores

for i in lista_materiales:
    validacion['e_'+ i] = ( (validacion['real_' + i] - validacion[i])/(validacion['real_'+i]+0.000001) ) 
    validacion['e_'+ i] = validacion['e_'+i].astype(float)

<ipython-input-96-c5688b49372a>:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  real = dfmateriales.groupby(['FechaEntrega','Planta'])['real_ADITIVO', 'real_ARENA', 'real_Agua', 'real_CEMENTO', 'real_CENIZA', 'real_FIBRA', 'real_GRAVA', 'real_OTROS'].sum().reset_index()


In [97]:
validacion.head().T

,0,1,2,3,4
FechaEntrega,2022-03-01 00:00:00,2022-03-01 00:00:00,2022-03-01 00:00:00,2022-03-01 00:00:00,2022-03-01 00:00:00
Planta,CO-PLANTA 240,CO-PLANTA BARRANQUILLA,CO-PLANTA BELLO,CO-PLANTA BOSA,CO-PLANTA CALI
real_ADITIVO,258298,591384,127863,195630,313798
real_ARENA,263713,439530,60958.5,150303,99940
real_Agua,24458,119342,11464,16758,26840
real_CEMENTO,71698,218591,32834,47387.4,72071
real_CENIZA,8839,0,5258,502,7454
real_FIBRA,0,0,0,0,0
real_GRAVA,203818,580693,121771,140235,231835
real_OTROS,0,0,0,0,0


## Actualización de datos en SQL

In [100]:
params = urllib.parse.quote_plus("DRIVER={ODBC Driver 17 for SQL Server};SERVER=USCLDBITVMP01;DATABASE=BI_Tableau;UID=usertableau;PWD=usertableau$")
engine = sa.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
        
validacion.to_sql("SCAC_AT41_materiales_realvsforecast", engine, index=False, if_exists="append", schema="dbo")

In [109]:
#Dataset desagregacion materias primas
comparacion = querySQL( "SELECT * FROM SCAC_AT41_materiales_realvsforecast WHERE Version = ?" , (version_forecast ) )
comparacion

,FechaEntrega,Planta,real_ADITIVO,real_ARENA,real_Agua,real_CEMENTO,real_CENIZA,real_FIBRA,real_GRAVA,real_OTROS,...,CEMENTO,CENIZA,GRAVA,Iteraciones,e_ADITIVO,e_ARENA,e_Agua,e_CEMENTO,e_CENIZA,e_GRAVA
0,2022-02-05,CO-PLANTA TULUA PARAJE LA RIVERA,30454.4,28826,4102.2,10063,1380,0,25243,0,...,8785.14,1074.58,29696.2,1_ciudad,-0.0424912,0.0088763,0.1621,0.126986,0.221315,-0.176413
1,2022-02-05,CO-PLANTA VISTA HERMOSA,77911.2,57759,4916,15967,0,0,44837,0,...,15955.1,1987.92,40501.7,1_ciudad,0.0765672,0.0221386,-0.298378,0.000745058,-1.98792e+09,0.0966893
2,2022-02-06,CO-PLANTA CUCUTA,53810.5,43483,4892,12216,1679,0,34491,0,...,0,0,0,1_ciudad,1,1,1,1,1,1
3,2022-02-06,CO-PLANTA NEIVA,347229,189718,19104,56842,5596,0,158362,0,...,0,0,0,1_ciudad,1,1,1,1,1,1
4,2022-02-06,CO-PLANTA SUMAPAZ,249281,115186,6540,44249,6271,0,66837,0,...,0,0,0,1_ciudad,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6745,2022-02-28,CO-PLANTA SUMAPAZ,449879,318305,51739.2,93060.3,11675,0,241936,3870,...,82280.2,13400.8,231164,50,0.168557,0.0556314,0.438635,0.115839,-0.147821,0.0445241
6746,2022-02-28,CO-PLANTA SUR (DTE),178018,148841,18460,42337,7664,0,109438,0,...,57581.8,6959.77,137186,50,-0.488033,-0.156343,-0.320007,-0.360083,0.0918885,-0.253548
6747,2022-02-28,CO-PLANTA TOCANCIPA,393673,166052,29006.7,99161.5,0,0,245653,0,...,79940.1,79663.1,200779,50,0.234254,-0.436772,0.106371,0.193839,-7.96631e+10,0.182671
6748,2022-02-28,CO-PLANTA TULUA PARAJE LA RIVERA,22863.4,30280,2962.27,7797,1059,0,25719,0,...,18917.5,2149.95,60357.2,50,-2.29903,-1.06059,-1.49866,-1.42625,-1.03017,-1.34679


In [110]:
comparacion['mse_aditivos'] = (comparacion['ADITIVO'] - comparacion['real_ADITIVO']) ** 2
comparacion['mse_grava'] = (comparacion['GRAVA'] - comparacion['real_GRAVA']) ** 2
comparacion['mse_cemento'] = (comparacion['CEMENTO'] - comparacion['real_CEMENTO']) ** 2
comparacion['mse_aditivos'] = comparacion['mse_aditivos'].astype(float)
comparacion['mse_grava'] = comparacion['mse_grava'].astype(float)
comparacion['mse_cemento'] = comparacion['mse_cemento'].astype(float)


In [111]:
df_rmse = comparacion.groupby(['Iteraciones'])['mse_cemento', 'mse_aditivos', 'mse_grava'].mean().reset_index()
df_rmse['rmse_cemento'] = df_rmse['mse_cemento'] ** (0.5)
df_rmse['rmse_aditivos'] = df_rmse['mse_aditivos'] ** (0.5)
df_rmse['rmse_grava'] = df_rmse['mse_grava'] ** (0.5)
df_rmse


<ipython-input-111-0aa4142d07cc>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_rmse = comparacion.groupby(['Iteraciones'])['mse_cemento', 'mse_aditivos', 'mse_grava'].mean().reset_index()


,Iteraciones,mse_cemento,mse_aditivos,mse_grava,rmse_cemento,rmse_aditivos,rmse_grava
0,1,4.034101e+08,1.109380e+10,2.837641e+09,20085.071016,105327.091816,53269.515971
1,10_ciudad,3.916964e+08,1.304846e+10,2.925319e+09,19791.320277,114229.861470,54086.220182
2,10_cluster,3.989306e+08,1.254961e+10,2.873467e+09,19973.247939,112025.030936,53604.732137
3,1_ciudad,3.919622e+08,1.320995e+10,2.925427e+09,19798.034131,114934.538584,54087.218576
4,1_cluster,3.982089e+08,1.265981e+10,2.865063e+09,19955.172885,112515.825103,53526.280760
5,2,2.110278e+10,1.117595e+10,2.843144e+09,145267.965290,105716.353237,53321.136980
6,5,4.594740e+08,1.122164e+10,2.837840e+09,21435.345767,105932.263142,53271.379536
7,50,4.526253e+08,1.113106e+10,2.841181e+09,21274.992596,105503.848499,53302.731276
8,5_ciudad,3.932027e+08,1.303243e+10,2.927325e+09,19829.338494,114159.683781,54104.756853
9,5_cluster,3.990964e+08,1.255861e+10,2.872923e+09,19977.398418,112065.191765,53599.652728
